In [40]:
# Imports and Web Scraping Functions
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def fetch_ngx_list():
    """Fetch NGX company list from african-markets.com"""
    url = "https://www.african-markets.com/en/stock-markets/ngse/listed-companies"
    
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    session.mount("http://", HTTPAdapter(max_retries=retries))
    session.mount("https://", HTTPAdapter(max_retries=retries))
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0.0.0 Safari/537.36"
    }
    
    r = session.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    
    soup = BeautifulSoup(r.text, "html.parser")
    table = soup.find("table")
    rows = table.find_all("tr")
    
    header_cells = rows[0].find_all(["th", "td"])
    headers = [cell.get_text(strip=True) for cell in header_cells]
    
    data = []
    for row in rows[1:]:
        cols = [td.get_text(strip=True) for td in row.find_all("td")]
        if cols:
            data.append(cols)
    
    df = pd.DataFrame(data, columns=headers)
    return df

def scrape_top_movers():
    """Scrape top gainers, losers, and trades from NGX official website"""
    url = "https://ngxgroup.com"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0.0.0 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        tables = soup.find_all("table")
        
        # Check if we have enough tables
        if len(tables) < 3:
            print(f"⚠️ Warning: Expected 3 tables but found {len(tables)}")
            return {
                "Top Gainers": pd.DataFrame(),
                "Top Losers": pd.DataFrame(),
                "Top Trades": pd.DataFrame()
            }
        
        movers = {}
        categories = ["Top Gainers", "Top Losers", "Top Trades"]
        
        for idx, category in enumerate(categories):
            try:
                table = tables[idx]
                rows = table.find_all("tr")
                
                if len(rows) == 0:
                    movers[category] = pd.DataFrame()
                    continue
                
                # Extract headers
                header_cells = rows[0].find_all(["th", "td"])
                headers = [cell.get_text(strip=True) for cell in header_cells]
                
                # Extract data
                data = []
                for row in rows[1:]:
                    cols = [td.get_text(strip=True) for td in row.find_all("td")]
                    if cols:
                        data.append(cols)
                
                movers[category] = pd.DataFrame(data, columns=headers) if data else pd.DataFrame()
            except Exception as e:
                print(f"⚠️ Error parsing {category}: {e}")
                movers[category] = pd.DataFrame()
        
        return movers
    
    except Exception as e:
        print(f"❌ Error scraping NGX website: {e}")
        return {
            "Top Gainers": pd.DataFrame(),
            "Top Losers": pd.DataFrame(),
            "Top Trades": pd.DataFrame()
        }

print("✅ Imports and scraping functions loaded")

✅ Imports and scraping functions loaded


## 🎯 Key Finding: African Markets Uses Stock CODES

The URL structure is: `https://www.african-markets.com/en/stock-markets/ngse/listed-companies/company?code=AFRINSURE`

**Problem:** We need the stock CODE (e.g., `AFRINSURE`), not the company name.  
**Solution:** Extract codes from the company list table or scrape them from the links.


In [41]:
# Extract stock codes from the company list table
def fetch_ngx_with_codes():
    """Fetch NGX company list including stock codes from href links"""
    url = "https://www.african-markets.com/en/stock-markets/ngse/listed-companies"
    
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    session.mount("http://", HTTPAdapter(max_retries=retries))
    session.mount("https://", HTTPAdapter(max_retries=retries))
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0.0.0 Safari/537.36"
    }
    
    r = session.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    
    soup = BeautifulSoup(r.text, "html.parser")
    table = soup.find("table")
    rows = table.find_all("tr")
    
    # Extract headers
    header_cells = rows[0].find_all(["th", "td"])
    headers = [cell.get_text(strip=True) for cell in header_cells]
    headers.append("Stock_Code")  # Add new column for stock code
    
    # Extract rows and stock codes
    data = []
    for row in rows[1:]:
        cols = [td.get_text(strip=True) for td in row.find_all("td")]
        
        # Extract stock code from link
        link = row.find("a")
        stock_code = None
        if link and 'href' in link.attrs:
            href = link['href']
            # Extract code from URL like "company?code=AFRINSURE"
            if '?code=' in href:
                stock_code = href.split('?code=')[1]
        
        if cols:
            cols.append(stock_code)
            data.append(cols)
    
    df = pd.DataFrame(data, columns=headers)
    return df

# Fetch updated data with stock codes
df_ngx_with_codes = fetch_ngx_with_codes()
print(f"✅ Fetched {len(df_ngx_with_codes)} companies with stock codes")
print("\nFirst 10 companies:")
print(df_ngx_with_codes[['Company', 'Stock_Code', 'Price', 'Sector']].head(10))


✅ Fetched 156 companies with stock codes

First 10 companies:
                       Company   Stock_Code  Price             Sector
0   African Alliance Insurance    AFRINSURE   0.20         Financials
1                    McNichols    MCNICHOLS   3.27     Consumer Goods
2  Multi-Trex Integrated Foods    MULTITREX   0.36     Consumer Goods
3    Livingtrust Mortgage Bank  LIVINGTRUST   3.45         Financials
4    Veritas Kapital Assurance   VERITASKAP   1.71         Financials
5          Abbey Mortgage Bank     ABBEYBDS   6.40         Financials
6                ABC Transport     ABCTRANS   4.10  Consumer Services
7                Academy Press      ACADEMY   7.30        Industrials
8            Africa Prudential     AFRIPRUD  14.80         Technology
9                    Afromedia    AFROMEDIA   0.24  Consumer Services


## 📊 **Verdict: Historical Data Not Available on African-Markets.com**

After testing multiple URL patterns, **african-markets.com does NOT provide free historical data**.

---

## ✅ **Your Options for NGX Historical Data:**

### **Option 1: Daily Collection (FREE & RECOMMENDED)** 
Start scraping daily and build your own history:
```python
# Save daily snapshot
timestamp = datetime.now().strftime("%Y-%m-%d")
df_ngx.to_csv(f"data/raw/ngx/{timestamp}_snapshot.csv")
```
- ✅ After 30 days → calculate moving averages
- ✅ After 90 days → volatility analysis
- ✅ Full control over data quality

### **Option 2: Yahoo Finance (PARTIAL)**
Try major NGX stocks with `.LG` suffix:
```python
import yfinance as yf
yf.Ticker("DANGCEM.LG").history(period="1y")
```
- ⚠️ Only works for ~10-20 major stocks
- ✅ Free and reliable

### **Option 3: Investing.com (COMPLEX)**
Web scrape with POST requests to their AJAX endpoint
- ⚠️ Requires reverse-engineering their API
- ⚠️ May get blocked/rate-limited

### **Option 4: Paid Data Providers**
- **NGX Data Portal** (official, expensive)
- **Bloomberg Terminal** 
- **Refinitiv/Eikon**

---

## 🎯 **My Recommendation:**

**Combine Option 1 + Option 2:**
1. Use Yahoo Finance for major stocks (Dangote, MTN, Zenith, GTB)
2. For all other stocks, start daily collection TODAY
3. After 30 days, you'll have enough historical data for analysis


# 🌍 African Markets API

Let's use the official `afrimarket` Python package to access African Markets stock data.

In [12]:
# Step 1: Install the afrimarket package
!pip install afrimarket

Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 3.3 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 8.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 504.3 kB/s eta 0:00:00:010m
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
  Attempting uninstall: lxml
    Found existing installation: lxml 6.0.1
    Uninstalling lxml-6.0.1:
      Successfully uninstalled lxml-6.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.13.3
    Uninstalling beautifulsoup

In [14]:
# Import afrimarket
import afrimarket as afm

# Check available markets
print("📍 Available Markets:")
print(afm.markets)
print()

# Initialize Nigerian Stock Exchange
ngx = afm.Exchange(market='ngx')
print("✅ NGX Exchange initialized")

📍 Available Markets:
{'Botswana Stock Exchange': 'bse', 'Bourse Régionale des Valeurs Mobilières': 'brvm', 'Ghana Stock Exchange': 'gse', 'Johannesburg Stock Exchange': 'jse', 'Lusaka Securities Exchange': 'luse', 'Malawi Stock Exchange': 'mse', 'Nairobi Securities Exchange': 'nse', 'Nigerian Stock Exchange': 'ngx', 'Uganda Securities Exchange': 'use', 'Zimbabwe Stock Exchange': 'zse'}

✅ NGX Exchange initialized


## 1️⃣ Get All NGX Listed Companies

In [56]:
# Get all listed companies using afrimarket API
df_afrimarket = ngx.get_listed_companies()

print(f"📊 Afrimarket NGX Companies: {len(df_afrimarket)} stocks")
print(f"Columns: {df_afrimarket.columns.tolist()}")
print(f"\nFirst 10 companies:")
print(df_afrimarket.head(10))

📊 Afrimarket NGX Companies: 148 stocks
Columns: ['Ticker', 'Name', 'Volume', 'Price', 'Change']

First 10 companies:
       Ticker                            Name      Volume    Price  Change
0    ABBEYBDS         Abbey Mortgage Bank Plc   1281348.0     6.00   -0.40
1    ABCTRANS          Associated Bus Company   2000427.0     4.51    0.41
2     ACADEMY                   Academy Press     69479.0     7.30    0.00
3  ACCESSCORP             Access Holdings Plc  14283991.0    23.00    2.00
4   AFRINSURE      African Alliance Insurance         NaN     0.20     NaN
5    AFRIPRUD           Africa Prudential Plc    213155.0    14.80    0.00
6   AFROMEDIA                   Afromedia Plc         NaN     0.24     NaN
7       AIICO             Aiico Insurance Plc  12733172.0     3.88    0.09
8  AIRTELAFRI               Airtel Africa Plc      1538.0  2270.00    0.00
9        ALEX  Aluminium Extrusion Industries   2685610.0    23.80    2.15


In [43]:
# Ensure we have the scraped data for comparison
# If df_ngx_with_codes doesn't exist, fetch it
if 'df_ngx_with_codes' not in globals():
    print("📥 Fetching scraped data from african-markets.com...")
    df_ngx_with_codes = fetch_ngx_with_codes()
    print(f"✅ Fetched {len(df_ngx_with_codes)} companies\n")

# Compare with afrimarket data
print("🔍 COMPARISON: Afrimarket vs Scraped Data\n")
print("="*60)

print(f"\n📌 Scraped Data (df_ngx_with_codes):")
print(f"   Total stocks: {len(df_ngx_with_codes)}")
print(f"   Columns: {df_ngx_with_codes.columns.tolist()}")

print(f"\n📌 Afrimarket Data (df_afrimarket):")
print(f"   Total stocks: {len(df_afrimarket)}")
print(f"   Columns: {df_afrimarket.columns.tolist()}")

print(f"\n📊 Data Overlap:")
# Check which tickers are in both datasets
scraped_tickers = set(df_ngx_with_codes['Stock_Code'].dropna())
afrimarket_tickers = set(df_afrimarket['Ticker'].dropna())

print(f"   Scraped unique tickers: {len(scraped_tickers)}")
print(f"   Afrimarket unique tickers: {len(afrimarket_tickers)}")
print(f"   In both datasets: {len(scraped_tickers & afrimarket_tickers)}")
print(f"   Only in scraped: {len(scraped_tickers - afrimarket_tickers)}")
print(f"   Only in afrimarket: {len(afrimarket_tickers - scraped_tickers)}")

# Show sample comparison
print("\n📋 Sample stocks from both datasets:")
sample_comparison = pd.DataFrame({
    'Ticker': list((scraped_tickers & afrimarket_tickers))[:5]
})

for ticker in sample_comparison['Ticker']:
    scraped_price = df_ngx_with_codes[df_ngx_with_codes['Stock_Code'] == ticker]['Price'].values
    afrimarket_price = df_afrimarket[df_afrimarket['Ticker'] == ticker]['Price'].values
    
    if len(scraped_price) > 0 and len(afrimarket_price) > 0:
        print(f"   {ticker}: Scraped={scraped_price[0]}, Afrimarket={afrimarket_price[0]}")

🔍 COMPARISON: Afrimarket vs Scraped Data


📌 Scraped Data (df_ngx_with_codes):
   Total stocks: 156
   Columns: ['Company', 'Sector', 'Price', '1D', 'YTD', 'M.Cap', 'Date', 'Stock_Code']

📌 Afrimarket Data (df_afrimarket):
   Total stocks: 148
   Columns: ['Ticker', 'Name', 'Volume', 'Price', 'Change']

📊 Data Overlap:
   Scraped unique tickers: 154
   Afrimarket unique tickers: 148
   In both datasets: 141
   Only in scraped: 13
   Only in afrimarket: 7

📋 Sample stocks from both datasets:
   MANSARD: Scraped=13.70, Afrimarket=13.7
   IMG: Scraped=32.00, Afrimarket=32.0
   UNITYBNK: Scraped=1.51, Afrimarket=1.51
   JULI: Scraped=8.06, Afrimarket=8.06
   MAYBAKER: Scraped=19.00, Afrimarket=19.0


## 2️⃣ Test Individual Stock - Get Historical Prices, Competitors, Valuation

In [44]:
# Test with a major stock - UBA (United Bank for Africa)
uba_stock = afm.Stock(ticker="uba", market="ngx")
print("📈 Testing UBA Stock\n")

# Get historical prices
print("1️⃣ Historical Prices:")
uba_prices = uba_stock.get_price()
print(f"   Total data points: {len(uba_prices)}")
print(f"   Date range: {uba_prices['Date'].min()} to {uba_prices['Date'].max()}")
print(f"\n   Last 5 prices:")
print(uba_prices.tail())

📈 Testing UBA Stock

1️⃣ Historical Prices:
   Total data points: 2190
   Date range: 2016-01-05 00:00:00 to 2025-12-31 00:00:00

   Last 5 prices:
           Date  Price
2185 2025-12-17  39.80
2186 2025-12-18  41.60
2187 2025-12-23  39.40
2188 2025-12-30  40.85
2189 2025-12-31  41.65


In [45]:
# Get growth and valuation
print("2️⃣ Growth & Valuation:")
uba_valuation = uba_stock.get_growth_and_valuation()
print(uba_valuation)

2️⃣ Growth & Valuation:
      Growth & Valuation Growth & Valuation.1
0     Earnings Per Share                  NaN
1    Price/Earning Ratio                  NaN
2     Dividend Per Share                  NaN
3         Dividend Yield                  NaN
4     Shares Outstanding                  41B
5  Market Capitalization                1.76T


In [27]:
# Get competitors
print("3️⃣ Competitors:")
uba_competitors = uba_stock.get_competitors()
print(f"   Found {len(uba_competitors)} competitors")
print(uba_competitors)

3️⃣ Competitors:
   Found 10 competitors
          Code                             Name M. Cap.  Close     YTD
0   ACCESSCORP              Access Holdings Plc   1.23T  23.00  +9.52%
1   ZENITHBANK                  Zenith Bank Plc   2.65T  64.50  +4.37%
2  FIRSTHOLDCO                 First HoldCo Plc   2.04T  48.80  +1.88%
3         GTCO           Guaranty Trust Holding   3.36T  92.30  +1.76%
4          ETI        Ecobank Transnational Inc    994B  41.90     NaN
5   FIDELITYBK                Fidelity Bank Plc    954B  19.00     NaN
6     MBENEFIT        Mutual Benefits Assurance   68.4B   3.41    +10%
7      DEAPCAP  Deap Capital Management & Trust   3.14B   2.09    +10%
8        WAPIC             Coronation Insurance     84B   3.50  +7.69%
9   VERITASKAP    Veritas Kapital Assurance Plc   25.4B   1.83  +7.02%


In [28]:
# Get market performance
print("4️⃣ Market Performance (Period):")
uba_performance_period = uba_stock.get_stock_market_performance_period()
print(uba_performance_period)

print("\n5️⃣ Market Performance (Last 10 Days):")
uba_performance_date = uba_stock.get_stock_market_performance_date()
print(uba_performance_date)

4️⃣ Market Performance (Period):
      1WK   4WK  3MO     6MO     1YR     YTD
0  +9.14%  +12%  NaN  +17.3%  +19.4%  +3.24%

5️⃣ Market Performance (Last 10 Days):
         Date    Volume  Close  Change Change%
0  2026-01-02   4344497  43.00    1.35  +3.24%
1  2025-12-31   5330631  41.65    0.80  +1.96%
2  2025-12-30   7805855  40.85   -0.15  -0.37%
3  2025-12-29  11637552  41.00   -1.00  -2.38%
4  2025-12-24   7196115  42.00    2.60  +6.60%
5  2025-12-23   7487158  39.40    0.40  +1.03%
6  2025-12-22  10188680  39.00   -1.00  -2.50%
7  2025-12-19   6602363  40.00   -1.60  -3.85%
8  2025-12-18   3713287  41.60    1.80  +4.52%
9  2025-12-17  12607707  39.80     NaN     NaN


## 3️⃣ Get Market Overview - Top Gainers, Losers, Index Price

In [29]:
# Get top gainers
print("📈 TOP GAINERS:")
top_gainers = ngx.get_top_gainers()
print(top_gainers)

📈 TOP GAINERS:
  Top Gainers (53)  Top Gainers (53).1 Top Gainers (53).2
0         FTNCOCOA                5.50            +10.00%
1         ABCTRANS                4.51            +10.00%
2         MBENEFIT                3.41            +10.00%
3          DEAPCAP                2.09            +10.00%
4             ALEX               23.80             +9.93%
5        AUSTINLAZ                4.67             +9.88%


In [30]:
# Get bottom losers
print("📉 BOTTOM LOSERS:")
bottom_losers = ngx.get_bottom_losers()
print(bottom_losers)

📉 BOTTOM LOSERS:
  Bottom Losers (10)  Bottom Losers (10).1 Bottom Losers (10).2
0           ABBEYBDS                  6.00               -6.25%
1               FCMB                 11.50               -4.56%
2             SEPLAT               5610.00               -3.43%
3          GUINEAINS                  1.30               -2.26%
4         UNIVINSURE                  1.19               -1.65%
5            CADBURY                 59.00               -1.50%


In [54]:
# Get index price history
print("📊 NGX INDEX PRICE HISTORY:")
index_prices = ngx.get_index_price()
print(f"Total data points: {len(index_prices)}")
print(f"Date range: {index_prices['Date'].min()} to {index_prices['Date'].max()}")
print(f"\nLast 10 index values:")
print(index_prices.tail(10))

📊 NGX INDEX PRICE HISTORY:
Total data points: 2450
Date range: 2016-01-05 00:00:00 to 2026-01-02 00:00:00

Last 10 index values:
           Date      Price
2440 2025-12-17  149842.82
2441 2025-12-18  150363.05
2442 2025-12-19  152057.38
2443 2025-12-22  152459.07
2444 2025-12-23  153354.14
2445 2025-12-24  153539.83
2446 2025-12-29  154389.53
2447 2025-12-30  155034.72
2448 2025-12-31  155613.03
2449 2026-01-02  156492.36


## 📊 Summary: Afrimarket API vs Scraped Data

**What we learned:**
1. ✅ Afrimarket provides comprehensive NGX data through a clean API
2. ✅ Historical prices available (10+ years of data)
3. ✅ Additional features: competitors, valuation metrics, performance tracking
4. 🔍 Compare data quality and coverage with our scraped approach

# 🏦 NGX Official Website Analysis

Let's perform the same analysis using the official NGX website data to compare with afrimarket.

## 1️⃣ Get All NGX Listed Companies (Official NGX Website)

In [48]:
# Get all listed companies from african-markets.com (which uses NGX data)
df_ngx = fetch_ngx_list()

print(f"📊 NGX Listed Companies: {len(df_ngx)} stocks")
print(f"Columns: {df_ngx.columns.tolist()}")
print(f"\nFirst 10 companies:")
print(df_ngx.head(10))

📊 NGX Listed Companies: 156 stocks
Columns: ['Company', 'Sector', 'Price', '1D', 'YTD', 'M.Cap', 'Date']

First 10 companies:
                       Company             Sector  Price      1D       YTD  \
0   African Alliance Insurance         Financials   0.20       -         -   
1                    McNichols     Consumer Goods   3.27  +9.00%  +103.11%   
2  Multi-Trex Integrated Foods     Consumer Goods   0.36       -         -   
3    Livingtrust Mortgage Bank         Financials   3.45  +0.88%   -21.23%   
4    Veritas Kapital Assurance         Financials   1.71  +6.21%   +25.74%   
5          Abbey Mortgage Bank         Financials   6.40       -  +113.33%   
6                ABC Transport  Consumer Services   4.10  +5.13%  +233.33%   
7                Academy Press        Industrials   7.30       -  +143.33%   
8            Africa Prudential         Technology  14.80  +6.09%   -27.98%   
9                    Afromedia  Consumer Services   0.24       -         -   

   M.Cap   Date

## 2️⃣ Get Market Movers from Official NGX Website

In [49]:
# Get top movers from ngxgroup.com
ngx_movers = scrape_top_movers()

if len(ngx_movers["Top Gainers"]) > 0:
    print("📈 TOP GAINERS (NGX Official):")
    print(ngx_movers["Top Gainers"])
    print()

    print("📉 TOP LOSERS (NGX Official):")
    print(ngx_movers["Top Losers"])
    print()

    print("💹 TOP TRADES (NGX Official):")
    print(ngx_movers["Top Trades"])
else:
    print("⚠️ Unable to fetch NGX official website data")
    print("The NGX website may have changed or is currently unavailable.")
    print("\nNote: Afrimarket API is still the recommended source as it's more reliable.")

⚠️ Unable to fetch NGX official website data
The NGX website may have changed or is currently unavailable.

Note: Afrimarket API is still the recommended source as it's more reliable.


## 3️⃣ Compare Data Sources

Let's compare the data from different sources:
- **Afrimarket API** - Uses kwayisi.org data
- **African-Markets.com** - Web scraping
- **NGX Official Website** - Direct from exchange

In [52]:
# Comparison of data sources
print("📊 DATA SOURCE COMPARISON\n")
print("="*70)

print(f"\n1️⃣ Afrimarket API:")
print(f"   Total stocks: {len(df_afrimarket)}")
print(f"   Data includes: Price, Volume, Change")
print(f"   Historical data: ✅ Yes (10+ years)")
print(f"   Ease of use: ✅ Very Easy (Python API)")

print(f"\n2️⃣ African-Markets.com (Web Scraping):")
print(f"   Total stocks: {len(df_ngx)}")
print(f"   Data includes: Company, Price, Sector, etc.")
print(f"   Historical data: ❌ No")
print(f"   Ease of use: ⚠️ Moderate (Web scraping)")

print(f"\n3️⃣ NGX Official Website:")
print(f"   Top Gainers: {len(ngx_movers['Top Gainers'])} stocks")
print(f"   Top Losers: {len(ngx_movers['Top Losers'])} stocks")
print(f"   Top Trades: {len(ngx_movers['Top Trades'])} stocks")
print(f"   Data includes: Symbol, Price, Change, Volume")
print(f"   Historical data: ❌ Requires advanced scraping")
print(f"   Ease of use: ⚠️ Moderate (Web scraping)")

# Sample price comparison for overlapping stocks
print("\n\n💰 PRICE COMPARISON (Sample stocks):\n")
print(f"{'Stock':<12} {'Afrimarket':<15} {'African-Mkts':<15} {'NGX Official':<15}")
print("-" * 60)

# Get sample stocks that appear in multiple sources
sample_stocks = ['UBA', 'ZENITHBANK', 'GTCO', 'DANGCEM', 'MTNN']

for stock in sample_stocks:
    afr_price = "N/A"
    web_price = "N/A"
    ngx_price = "N/A"
    
    # Get from afrimarket
    afr_row = df_afrimarket[df_afrimarket['Ticker'].str.upper() == stock.upper()]
    if len(afr_row) > 0:
        afr_price = str(afr_row.iloc[0]['Price'])
    
    # Get from african-markets.com scrape
    # Match by company name (not ticker)
    company_mapping = {
        'UBA': 'United Bank',
        'ZENITHBANK': 'Zenith',
        'GTCO': 'Guaranty',
        'DANGCEM': 'Dangote Cement',
        'MTNN': 'MTN'
    }
    
    if stock in company_mapping:
        web_row = df_ngx[df_ngx['Company'].str.contains(company_mapping[stock], case=False, na=False)]
        if len(web_row) > 0:
            web_price = str(web_row.iloc[0]['Price'])
    
    # Get from NGX official (check gainers/losers/trades) - only if data is available
    for category in ['Top Gainers', 'Top Losers', 'Top Trades']:
        if len(ngx_movers[category]) > 0 and 'Symbol' in ngx_movers[category].columns:
            ngx_row = ngx_movers[category][ngx_movers[category]['Symbol'].str.upper() == stock.upper()]
            if len(ngx_row) > 0:
                ngx_price = str(ngx_row.iloc[0]['Price'])
                break
    
    print(f"{stock:<12} {afr_price:<15} {web_price:<15} {ngx_price:<15}")

📊 DATA SOURCE COMPARISON


1️⃣ Afrimarket API:
   Total stocks: 148
   Data includes: Price, Volume, Change
   Historical data: ✅ Yes (10+ years)
   Ease of use: ✅ Very Easy (Python API)

2️⃣ African-Markets.com (Web Scraping):
   Total stocks: 156
   Data includes: Company, Price, Sector, etc.
   Historical data: ❌ No
   Ease of use: ⚠️ Moderate (Web scraping)

3️⃣ NGX Official Website:
   Top Gainers: 0 stocks
   Top Losers: 0 stocks
   Top Trades: 0 stocks
   Data includes: Symbol, Price, Change, Volume
   Historical data: ❌ Requires advanced scraping
   Ease of use: ⚠️ Moderate (Web scraping)


💰 PRICE COMPARISON (Sample stocks):

Stock        Afrimarket      African-Mkts    NGX Official   
------------------------------------------------------------
UBA          43.0            41.65           N/A            
ZENITHBANK   64.5            61.80           N/A            
GTCO         92.3            90.70           N/A            
DANGCEM      609.0           609.00          N/A    

## 📊 Final Verdict: Best Data Source for NGX Stocks

### 🏆 **Winner: Afrimarket API**

**Reasons:**
1. ✅ **Historical Data**: 10+ years of price history
2. ✅ **Easy Integration**: Clean Python API
3. ✅ **Comprehensive**: Price, volume, valuation, competitors
4. ✅ **Reliable**: Uses kwayisi.org as backend
5. ✅ **No Scraping**: No need to worry about website changes

### 📌 **When to use other sources:**
- **African-Markets.com**: When you need sector classification
- **NGX Official**: For real-time top movers and official announcements
- **Yahoo Finance**: For major stocks with `.LG` suffix (DANGCEM.LG)

### 🎯 **Recommendation for Stock Pipeline:**
Use **Afrimarket API** as primary data source:
```python
import afrimarket as afm

# Get exchange data
ngx = afm.Exchange(market='ngx')
stocks = ngx.get_listed_companies()

# Get individual stock data
stock = afm.Stock(ticker='uba', market='ngx')
prices = stock.get_price()  # Historical prices
competitors = stock.get_competitors()
valuation = stock.get_growth_and_valuation()
```

In [62]:
# Save today's data to CSV
import os

# Create directory if it doesn't exist - use archive/data instead
data_dir = "/home/Stock_pipeline/archive/data/raw"
os.makedirs(data_dir, exist_ok=True)

# Save snapshots
timestamp = datetime.now().strftime("%Y-%m-%d")

# Save all listed companies with current prices
csv_path = f"{data_dir}/{timestamp}_all_stocks.csv"
df_afrimarket.to_csv(csv_path, index=False)
print(f"✅ Saved all stocks to: {csv_path}")

# Save top gainers
gainers_path = f"{data_dir}/{timestamp}_top_gainers.csv"
top_gainers.to_csv(gainers_path, index=False)
print(f"✅ Saved top gainers to: {gainers_path}")

# Save top losers
losers_path = f"{data_dir}/{timestamp}_top_losers.csv"
bottom_losers.to_csv(losers_path, index=False)
print(f"✅ Saved top losers to: {losers_path}")

# Save index price history
index_path = f"{data_dir}/{timestamp}_index_history.csv"
index_prices.to_csv(index_path, index=False)
print(f"✅ Saved index history to: {index_path}")

print(f"\n📁 All data saved to: {data_dir}")

✅ Saved all stocks to: /home/Stock_pipeline/archive/data/raw/2026-01-04_all_stocks.csv
✅ Saved top gainers to: /home/Stock_pipeline/archive/data/raw/2026-01-04_top_gainers.csv
✅ Saved top losers to: /home/Stock_pipeline/archive/data/raw/2026-01-04_top_losers.csv
✅ Saved index history to: /home/Stock_pipeline/archive/data/raw/2026-01-04_index_history.csv

📁 All data saved to: /home/Stock_pipeline/archive/data/raw


In [59]:
from datetime import datetime

# Get today's date
today = datetime.now().strftime("%Y-%m-%d")
print(f"📅 Market Data as of {today}\n")
print("="*70)

# 1. Today's Index Price
print("\n📊 NGX ALL-SHARE INDEX:")
latest_index = index_prices.tail(1)
print(f"   Latest Index Value: {latest_index['Price'].values[0]:,.2f}")
print(f"   Date: {latest_index['Date'].values[0]}")

# 2. Top Gainers Today
print("\n📈 TOP GAINERS:")
print(top_gainers.head(10))

# 3. Top Losers Today
print("\n📉 TOP LOSERS:")
print(bottom_losers.head(10))

# 4. Major Stocks Current Prices
print("\n💰 MAJOR STOCKS - CURRENT PRICES:")
major_stocks = ['UBA', 'ZENITHBANK', 'GTCO', 'DANGCEM', 'MTNN', 'NESTLE', 'AIRTELAFRI']

major_stocks_data = []
for ticker in major_stocks:
    stock_info = df_afrimarket[df_afrimarket['Ticker'].str.upper() == ticker.upper()]
    if len(stock_info) > 0:
        major_stocks_data.append({
            'Ticker': ticker,
            'Name': stock_info.iloc[0]['Name'],
            'Price': stock_info.iloc[0]['Price'],
            'Change': stock_info.iloc[0]['Change'],
            'Volume': stock_info.iloc[0]['Volume']
        })

df_major = pd.DataFrame(major_stocks_data)
print(df_major.to_string(index=False))

print("\n" + "="*70)

📅 Market Data as of 2026-01-04


📊 NGX ALL-SHARE INDEX:
   Latest Index Value: 156,492.36
   Date: 2026-01-02T00:00:00.000000000

📈 TOP GAINERS:
  Top Gainers (53)  Top Gainers (53).1 Top Gainers (53).2
0         FTNCOCOA                5.50            +10.00%
1         ABCTRANS                4.51            +10.00%
2         MBENEFIT                3.41            +10.00%
3          DEAPCAP                2.09            +10.00%
4             ALEX               23.80             +9.93%
5        AUSTINLAZ                4.67             +9.88%

📉 TOP LOSERS:
  Bottom Losers (10)  Bottom Losers (10).1 Bottom Losers (10).2
0           ABBEYBDS                  6.00               -6.25%
1               FCMB                 11.50               -4.56%
2             SEPLAT               5610.00               -3.43%
3          GUINEAINS                  1.30               -2.26%
4         UNIVINSURE                  1.19               -1.65%
5            CADBURY                 59.00        

# 📅 Today's Market Data from Afrimarket API